In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec  8 10:51:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    35W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install sigopt

Hugging face log in

In [6]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


### Load dataset
!!! Only run this part once
Training will be done with the NST dataset and evaluated agains the Mozzila one

This is done so we can compare the two models and see if there is an imporvement
from using a diffrend data source.

In [23]:
from datasets import load_dataset, DatasetDict, load_dataset_builder

nst = DatasetDict()

nst["train"] = load_dataset("jzju/nst", 'speech', split="train", use_auth_token=True)
# common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "sv-SE", split="train+validation", use_auth_token=True)
nst["test"] = load_dataset("jzju/nst", 'speech', split="test", use_auth_token=True)

KeyboardInterrupt: ignored

Since the dataset usef or training is huge – only use a part of it

In [ ]:
nst["train"]=nst["train"].select(range(23000))
nst["valid"]=nst["test"].select(range(2300))
nst["test"]=nst["test"].select(range(2301, 7200))
print(nst)

Clean up the colums

In [ ]:
# column names must match in train and test
nst["train"] = nst["train"].rename_column("text", "sentence")
nst["test"] = nst["test"].rename_column("text", "sentence")
nst["valid"] = nst["valid"].rename_column("text", "sentence")

print(nst)

### Create tokenizers, feature extractor

In [7]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [8]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Swedish", task="transcribe")

In [10]:
print(nst["train"][0])

{'audio': {'bytes': b'RIFF\xe4\xc2\x04\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00\x80>\x00\x00\x00}\x00\x00\x02\x00\x10\x00data\xc0\xc2\x04\x00\x8f\xff\x9c\xff\xb0\xff\xee\xff\x00\x00\xfe\xff\x0e\x00\t\x00\x01\x00\x08\x00\n\x00\x0e\x00\x03\x00\xff\xff\x04\x00\x08\x00\x0b\x00\x06\x00\x01\x00\x06\x00\x06\x00\xfd\xff\x0e\x00\xfb\xff\x00\x00\xff\xff\t\x00\x01\x00\x0f\x00\n\x00\x01\x00\x13\x00\x0e\x00\x06\x00\x05\x00\n\x00\x08\x00\xf8\xff\x00\x00\x07\x00\x08\x00\xfd\xff\xfc\xff\xfb\xff\xf7\xff\x05\x00\x02\x00\x04\x00\x01\x00\x03\x00\x04\x00\x07\x00\x06\x00\x02\x00\x04\x00\x05\x00\xf8\xff\xf9\xff\x06\x00\x03\x00\xf4\xff\x12\x00\x0c\x00\x01\x00\x10\x00\x13\x00\x00\x00\x16\x00\x0f\x00\x07\x00\xff\xff\n\x00\x06\x00\xf7\xff\x06\x00\x05\x00\x06\x00\xff\xff\x07\x00\x01\x00\x06\x00\x05\x00\x00\x00\n\x00\xf8\xff\t\x00\x03\x00\x01\x00\x02\x00\t\x00\r\x00\x0b\x00\r\x00\x01\x00\xff\xff\x05\x00\t\x00\x00\x00\xf3\xff\xfd\xff\xfe\xff\xff\xff\xf9\xff\x0e\x00\x08\x00\xff\xff\x08\x00\n\x00\x0e\x00\x0b\x00\x

### Prepare the dataset

In [11]:
from datasets import Audio

nst = nst.cast_column("audio", Audio(sampling_rate=16000))

Helper function to prepare the dataset

In [12]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

Do the preparation

In [13]:
nst = nst.map(prepare_dataset, remove_columns=nst.column_names["train"], num_proc=2)

#1:   0%|          | 0/12000 [00:00<?, ?ex/s]

#0:   0%|          | 0/12000 [00:00<?, ?ex/s]

#0:   0%|          | 0/2400 [00:00<?, ?ex/s]

#1:   0%|          | 0/2399 [00:00<?, ?ex/s]

#0:   0%|          | 0/1200 [00:00<?, ?ex/s]

#1:   0%|          | 0/1200 [00:00<?, ?ex/s]

### Save the data to drive

In [16]:
nst.push_to_hub("fimster/NST_small_whisper", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/47 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/10 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/5 [00:00<?, ?it/s]

In [14]:
from google.colab import drive

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [15]:
nst["train"].save_to_disk("gdrive/MyDrive/ID2223/Lab 2 NST/train")
nst["test"].save_to_disk("gdrive/MyDrive/ID2223/Lab 2 NST/test")
nst["valid"].save_to_disk("gdrive/MyDrive/ID2223/Lab 2 NST/validate")

### Load the data from drive 
Avoid downloading the dataset again and process it

In [9]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
from datasets import load_from_disk, DatasetDict

nst = DatasetDict()

nst["train"] = load_from_disk("drive/MyDrive/ID2223/Lab 2 NST/train")
nst["test"] = load_from_disk("drive/MyDrive/ID2223/Lab 2 NST/test")
nst["valid"] = load_from_disk("drive/MyDrive/ID2223/Lab 2 NST/validate")

In [11]:
print(nst)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 24000
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 4799
    })
    valid: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 2400
    })
})


### Load the dataset from HF

In [ ]:
from datasets import load_dataset, DatasetDict

nst = DatasetDict()

nst["train"] = load_dataset("fimster/NST_small_whisper", "train", use_auth_token=True)
nst["test"] = load_dataset("fimster/NST_small_whisper", "test", use_auth_token=True)
nst["valid"] = load_dataset("fimster/NST_small_whisper", "valid", use_auth_token=True)

In [ ]:
print(nst)

### Model stuff

Create processor

In [12]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Swedish", task="transcribe")

Create helper class

In [13]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [14]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

Create evalation metrics

In [15]:
import evaluate

metric = evaluate.load("wer")

In [16]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Load the model

In [17]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [18]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

### Training

Training arguments

In [19]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="drive/MyDrive/ID2223/Lab 2 NST/model/whisper-small-sv-SE-NST",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    save_strategy = "steps",
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

In [20]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=nst["train"],
    eval_dataset=nst["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

Cloning https://huggingface.co/fimster/whisper-small-sv-SE-NST into local empty directory.
max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


Test the save

In [21]:
processor.save_pretrained(training_args.output_dir)

Feature extractor saved in drive/MyDrive/ID2223/Lab 2 NST/model/whisper-small-sv-SE-NST/preprocessor_config.json
tokenizer config file saved in drive/MyDrive/ID2223/Lab 2 NST/model/whisper-small-sv-SE-NST/tokenizer_config.json
Special tokens file saved in drive/MyDrive/ID2223/Lab 2 NST/model/whisper-small-sv-SE-NST/special_tokens_map.json
added tokens file saved in drive/MyDrive/ID2223/Lab 2 NST/model/whisper-small-sv-SE-NST/added_tokens.json


Actauall training

In [22]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 24000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4000
  Number of trainable parameters = 241734912
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
1000,0.163500,0.169413,13.499323
2000,0.070000,0.143108,11.380244
3000,0.059700,0.130233,10.468200
4000,0.019300,0.130549,10.167794


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c

TrainOutput(global_step=4000, training_loss=0.1840304369330406, metrics={'train_runtime': 33728.0347, 'train_samples_per_second': 1.898, 'train_steps_per_second': 0.119, 'total_flos': 1.846946562048e+19, 'train_loss': 0.1840304369330406, 'epoch': 2.67})

Upload to HF

In [27]:
kwargs = {
    "dataset_tags": "NST",
    "dataset": "NST Swedish ASR",  # a 'pretty' name for the training dataset
    "dataset_args": "config: speech, split: test",
    "language": "sv",
    "model_name": "Whisper Small sv-SE NST - Lab 2",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "i-dont-know-what-im-doing",
}

In [28]:
trainer.push_to_hub(**kwargs)

Saving model checkpoint to drive/MyDrive/ID2223/Lab 2 NST/model/whisper-small-sv-SE-NST
Configuration saved in drive/MyDrive/ID2223/Lab 2 NST/model/whisper-small-sv-SE-NST/config.json
Model weights saved in drive/MyDrive/ID2223/Lab 2 NST/model/whisper-small-sv-SE-NST/pytorch_model.bin
Feature extractor saved in drive/MyDrive/ID2223/Lab 2 NST/model/whisper-small-sv-SE-NST/preprocessor_config.json
